In [5]:
from src.run_model import PEFTModel
from src.peft_search_space import PEFTSearchSpace
from src.dataset_wrapper import PEFTDataset
from pruning_methods import prune_model

import argparse
import logging
import random
import time

logger = logging.getLogger('controller')
logger.setLevel(logging.INFO)  # 设置日志级别
file_handler = logging.FileHandler('output.log', mode='w')
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

logger.info('Start loading dataset')
dataset = PEFTDataset(
    'rotten_tomatoes', instructs=True, test_size=0.2).get_dataset()

# parser = argparse.ArgumentParser()
# parser.add_argument('--lora', type=int, nargs='+')
# parser.add_argument('--adapter', type=int, nargs='+')
# parser.add_argument('--base_lora', type=int)
# parser.add_argument('--base_adapter', type=int)
args = parser.parse_args(args=[])
# args={}

searched_args = []
searched_res = []

# baseline
logger.info('Start baseline')

print(dataset["train"]['text'][0])


def save_gradients(model):
    gradients = {}
    for name, param in model.model.named_parameters():
        if param.grad is not None:
            print(param.grad)
            gradients[name] = param.grad.clone().cpu().numpy()
    return gradients






search_list = [32] * 12
args.lora = search_list
args.epochs = 1
args.instructs = 1
# args.adapter = search_list
configs = PEFTSearchSpace(args).get_config()
model = PEFTModel(configs, dataset)
res = model.run()
grad = save_gradients(model)
logger.info(f'Result for {search_list}: {res}')
logger.info(f'grad: {grad}')
print(grad)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 852
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 214
    })
})
beautifully directed and convincingly acted .


/home/autopeft/anaconda3/envs/autopeft/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.00it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Map: 100%|██████████| 852/852 [00:00<00:00, 13684.57 examples/s]


{'text': ['beautifully directed and convincingly acted .', 'death to smoochy tells a moldy-oldie , not-nearly -as-nasty -as-it- thinks-it-is joke . over and over again .', 'has not so much been written as assembled , frankenstein-like , out of other , marginally better shoot-em-ups .', "as his circle of friends keeps getting smaller one of the characters in long time dead says 'i'm telling you , this is f * * * ed' . maybe he was reading the minds of the audience .", "director roger kumble offers just enough sweet and traditional romantic comedy to counter the crudity . and there's the inimitable diaz , holding it all together .", 'spectators will indeed sit open-mouthed before the screen , not screaming but yawning .', 'some writer dude , i think his name was , uh , michael zaidan , was supposed to have like written the screenplay or something , but , dude , the only thing that i ever saw that was written down were the zeroes on my paycheck .', 'a film that presents an interesting , e

Map: 100%|██████████| 214/214 [00:00<00:00, 11070.04 examples/s]
/home/autopeft/anaconda3/envs/autopeft/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


{'text': ["mazel tov to a film about a family's joyous life acting on the yiddish stage .", 'less cinematically powerful than quietly and deeply moving , which is powerful in itself .', 'this is very much of a mixed bag , with enough negatives to outweigh the positives .', 'missteps take what was otherwise a fascinating , riveting story and send it down the path of the mundane .', "while it's nothing we haven't seen before from murphy , i spy is still fun and enjoyable and so aggressively silly that it's more than a worthwhile effort .", 'it appears to have been modeled on the worst revenge-of-the-nerds clichés the filmmakers could dredge up .', "it's all arty and jazzy and people sit and stare and turn away from one another instead of talking and it's all about the silences and if you're into that , have at it .", "if it's unnerving suspense you're after -- you'll find it with ring , an indisputably spooky film ; with a screenplay to die for .", "even when he's not at his most critica

/home/autopeft/anaconda3/envs/autopeft/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


{'eval_loss': nan, 'eval_accuracy': 0.48598130841121495, 'eval_f1': 0.3178745665079645, 'eval_final_score': 0.4019279374595897, 'eval_runtime': 4.6099, 'eval_samples_per_second': 46.422, 'eval_steps_per_second': 0.868, 'epoch': 1.0}
{}


In [6]:
import datasets


dataset_train=datasets.load_dataset('rotten_tomatoes',split="train[:100]")
dataset_test=datasets.load_dataset('rotten_tomatoes',split="test[:50%]")
dataset=datasets.DatasetDict({'train':dataset_train,'test':dataset_test})

def add_prefix(example):
    example["text"] = 'This is an prefix' + example["text"]
    return example

instructed_dataset=dataset.map(add_prefix)

print(instructed_dataset)
instructed_dataset['train']['text'][:10]


Map: 100%|██████████| 533/533 [00:00<00:00, 48814.64 examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 533
    })
})


['This is an prefixthe rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'This is an prefixthe gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .',
 'This is an prefixeffective but too-tepid biopic',
 'This is an prefixif you sometimes like to go to the movies to have fun , wasabi is a good place to start .',
 "This is an prefixemerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one .",
 'This is an prefixthe film provides some great insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game .',
 'This is an prefixoffers that rare combination of entertainment and education .',
 'This is 